In [2]:
import cohere
import pandas as pd
import faiss
from sentence_transformers import SentenceTransformer

e:\GraduationProject\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df = pd.read_csv("plant_diseases_treatment.csv")

In [5]:
# Combine text
def combine_columns(row):
    return f"النبات: {row['اسم النبات']}, المرض: {row['اسم المرض']}, العلاج: {row['العلاج']}, طريقة الرش: {row['طريقة الرش']}, توقيت الرش: {row['توقيت الرش']}, إجراءات إضافية: {row['إجراءات إضافية']}"

texts = df.apply(combine_columns, axis=1).tolist()

In [7]:
# Embedding model
embedding_model = SentenceTransformer("BAAI/bge-small-en-v1.5") 
embeddings = embedding_model.encode(texts, show_progress_bar=True)


Batches: 100%|██████████| 1/1 [00:01<00:00,  1.75s/it]


In [8]:
# Create FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

In [ ]:
# Search function
#def retrieve_answer(question):
    #query_vector = embedding_model.encode([question])
   # D, I = index.search(query_vector, k=1)
  #  return texts[I[0][0]]

In [37]:
def extract_crop_from_question(question):
    for crop in df["اسم النبات"].unique():
        if crop in question:
            return crop
    return None

def retrieve_answer(question):
    crop = extract_crop_from_question(question)
    if crop is None:
        return "لا يمكن تحديد اسم النبات من السؤال."

    filtered_df = df[df["اسم النبات"] == crop]
    if filtered_df.empty:
        return "لا توجد أمراض لهذا النبات في البيانات."

    texts = filtered_df.apply(combine_columns, axis=1).tolist()
    filtered_embeddings = embedding_model.encode(texts)

    temp_index = faiss.IndexFlatL2(filtered_embeddings.shape[1])
    temp_index.add(filtered_embeddings)

    query_vector = embedding_model.encode([question])
    D, I = temp_index.search(query_vector, k=3)  # ✅ زودنا عدد النتائج

    retrieved_texts = [texts[i] for i in I[0]]
    return " ".join(retrieved_texts)  # ✅ جمعهم كلهم في نص واحد

In [10]:
co = cohere.Client("0Qi0p8wQzYzRkXND2ROae9kILPwQOXECDosUqxEU")  # ← استبدل بـ API key بتاعك


In [33]:
def generate_response(question):
    retrieved_context = retrieve_answer(question)
    prompt = f"""
السؤال: {question}
المعلومات: {retrieved_context}
أجب بدقة بجملة واحدة فقط دون إضافة أي معلومات خارجية.
"""
    response = co.generate(
        model="command-r-plus",
        prompt=prompt,
        max_tokens=100,
        temperature=0.3,
    )
    return response.generations[0].text.strip()


In [19]:
print("الرد:", generate_response("ما هو علاج مرض جرب التفاح ؟"))


الرد: كابتان، مانكوزيب، ثيوفانات-ميثيل، مايكلوبوتانيل


In [34]:
print("الرد:", generate_response("ما هو توقيت الرش  مرض جرب التفاح ؟"))


الرد: كل 7-10 أيام


In [38]:
print("الرد:", generate_response("ما هي امراض الطماطم ؟"))


الرد: من أمراض الطماطم: العفن المبكر والبقعة البكتيرية.


In [39]:
print("الرد:", generate_response("ما هي امراض التفاح ؟"))


الرد: أمراض التفاح هي: العفن الأسود، وصدأ الأرز والتفاح، والجرب.


In [40]:
print("الرد:", generate_response("ما هي امراض البطاطس  ؟"))


الرد: أمراض البطاطس تشمل العفن المبكر والمتأخر، ويمكن علاجها بمبيدات فطرية وتحسين التهوية وإزالة الأوراق المصابة.


In [41]:
print("الرد:", generate_response("ما هي الاجراءات الاضافيه لمرض جرب التفاح ؟"))


الرد: إزالة الأجزاء المصابة.
